# Innovation : Convertir un audio wave en irm :

In [156]:
#importation des bibliothéques :
import wave 
import numpy as np
import math
import sounddevice as sd
from pydub.playback import play 
from pydub import AudioSegment 
import tkinter
from tkinter.filedialog import askopenfilename
import os

In [157]:
#La fonction qui prépare les données à compresser et  qui retourne les informations qu'on a besoin pour les compresser:
def préparation_data_wave(file):
    wave_read=wave.open(file,'r')
    nframes=wave_read.getnframes()
    content=wave_read.readframes(nframes)
    nchannels=wave_read.getnchannels()
    sampwidth=wave_read.getsampwidth()
    framerate=wave_read.getframerate()
    bitsPerSample=sampwidth*8
    taille_wave = os.path.getsize(file)
    audio_as_np_int32 = np.frombuffer(content, dtype=np.int32)
    samples = audio_as_np_int32.astype(np.float32)
    mn=min(samples)
    k=[]
    for i in samples:
        if i>0:
            k.append(i)
        else:
            k.append(i-mn)
    mx=max(k)
    k=(k/max(k))*255   
    données=[]
    for i in k :
        données.append(int(np.round(i)))
    strg=""
    for i in données :
        strg+=chr(int(i))

    return strg,audio_as_np_int32,framerate, bitsPerSample, sampwidth, nchannels,mn,mx,taille_wave
    

In [158]:
#La fonction qui compresse les données de l'audio en utilisant LZW:
def compress(data):
    """Compress a string to a list of output symbols."""
    # Build the dictionary.
    dict_size = 256
    dictionary = dict((chr(i), i) for i in range(dict_size))
    # in Python 3: dictionary = {chr(i): i for i in range(dict_size)}
    w = ""
    result = []
    for c in data:
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            result.append(dictionary[w])
            # Add wc to the dictionary.
            dictionary[wc] = dict_size
            dict_size += 1
            w = c
    # Output the code for w.
    if w:
        result.append(dictionary[w])
    return result


In [159]:
#la fonction qui retourne l'entête de fichier.irm
def genHeader(framerate, bitsPerSample, sampwidth, nchannels):
    datasize = sampwidth * nchannels * bitsPerSample // 8
    o = bytes("RIFF",'ascii')                                               # (4byte) Marks file as RIFF
    o += (datasize + 36).to_bytes(4,'little')                               # (4byte) File size in bytes excluding this and RIFF marker
    o += bytes("IRM",'ascii')                                              # (4byte) File type
    o += bytes("fmt ",'ascii')                                              # (4byte) Format Chunk Marker
    o += (16).to_bytes(4,'little')                                          # (4byte) Length of above format data
    o += (0).to_bytes(2,'little')                                           # (2byte) Format type (1 - PCM)
    o += (nchannels).to_bytes(nchannels,'little')                                    # (2byte)
    o += (framerate).to_bytes(4,'little')                                  # (4byte)
    o += (framerate * nchannels * bitsPerSample // 8).to_bytes(4,'little')  # (4byte)
    o += (nchannels * bitsPerSample // 8).to_bytes(2,'little')               # (2byte)
    o += (bitsPerSample).to_bytes(2,'little')                             # (2byte)
    o += bytes("min",'ascii')                                              # (4byte) File type
    o += bytes(str(mn),'utf-8')
    o += bytes("max",'ascii')                                              # (4byte) File type
    o += bytes(str(mx),'utf-8')    
    o += bytes("data",'ascii')                                              # (4byte) Data Chunk Marker
    o += (datasize).to_bytes(4,'little')                                    # (4byte) Data size in bytes
    return o

In [160]:
#la fonction qui crée le fichier .irm et retourne sa taille :
def compression_irm(compress,framerate, bitsPerSample, sampwidth, nchannels,mn,mx):
    #compress=compress(strg)
    données_compressées=""
    for i in range(len(compress)):
        données_compressées+=str(compress[i])
        if i!=(len(compress)-1):
            données_compressées+=","
    obj = open("audioirm.irm","wb")
    header=genHeader(framerate, bitsPerSample, sampwidth, nchannels)
    obj.write(header)
    obj.write(bytes(données_compressées,'utf-8'))
    taille_irm = os.path.getsize("audioirm.irm")
    return taille_irm

    

In [161]:
#La fonction qui fait l'opération inverse de la préparation des données :
def retour2(liste,z,y):
    li=[]
    for element in liste:
        e=element
        elem=(e/255)*int(y)
        if ((elem+int(z))<=0):
            li.append(elem)
        else:
            li.append(elem+int(z))
    return li

In [162]:
#La fonction de décodage LZW :
def decompress_lzw(seq):
    dictionaire={}
    decompresse=""
    decompresse+=chr(seq[0])
    i=1
    cmpt=256
    dictionaire[cmpt]=chr(seq[0])+chr(seq[1])
    while i<len(seq):
        if seq[i] <= 255:
            if seq[i-1]<=255: 
                dictionaire[cmpt]=chr(seq[i-1])+chr(seq[i])
            else:
                dictionaire[cmpt]=dictionaire[seq[i-1]]+chr(seq[i]) 
            decompresse+=chr(seq[i])
            cmpt+=1
            i=i+1
        else:
            
            if seq[i-1]<=255: 
                dictionaire[cmpt]=chr(seq[i-1])+chr(seq[i-1])
                dictionaire[cmpt]=chr(seq[i-1])+dictionaire[seq[i]][:1]
               
            else:
                dictionaire[cmpt]=dictionaire[seq[i-1]]+dictionaire[seq[i-1]]
                dictionaire[cmpt]=dictionaire[seq[i-1]]+dictionaire[seq[i]][:1]
            decompresse+=dictionaire[seq[i]]
            i+=1
            cmpt+=1
    return decompresse

In [163]:
#La fonction qui fait la décompression de l'audio .irm :
def decompression_irm():
    f= open("audio.irm","rb")
    octets=f.read()
    pos_donn=octets.find(bytes("data",'utf-8'))
    données=octets[pos_donn+8:]
    données_irm=(((str(données))[2:-1]).split(','))
    pos_min=octets.find(bytes("min",'utf-8'))
    pos_max=octets.find(bytes("max",'utf-8'))
    minimum=(((str(octets[pos_min+3:pos_max]))[2:-1]).split(','))[0]
    maximum=(((str(octets[pos_max+3:pos_donn]))[2:-1]).split(','))[0]
    maximum=int(float(maximum))
    minimum=int(float(minimum))
    comp=[]
    for i in range(0,len(données_irm)) :
        comp.append(int(données_irm[i]))
    decompress=decompress_lzw(comp)
    liste_decomp=[]
    for i in decompress:
        liste_decomp.append(ord(i))
    r=retour2(liste_decomp,minimum,maximum)
    return r


In [164]:
#La fonction qui permet d'écouter l'audio .irm :
def écouter_irm(données_décompressées,framerate):
    y= np.asarray(données_décompressées)
    sd.play(y,framerate)

In [165]:
#La fonction qui permet d'écouter l'audio original .wave :
def écouter_wave(données_wave,framerate):
    sd.play(données_wave,framerate)

In [166]:
#La fonction qui calcule le taux de compression
def taux_compression(taille_wave,taille_irm):
    T=1-(taille_irm/taille_wave)
    return (np.round(T*100,2))

In [169]:
#La fonction aqui permet d'afficher le repertoire où il ya les fichiers .wav et charger un choisi :
def explort(repertoireint):
    root=tkinter.Tk()
    filename=tkinter.filedialog.askopenfilename(initialdir=repertoireint,title="choisir le fichier",filetypes=(("tous type de fichier","*.*"),("fihiers wav","*.wav"),("fichier mp3","*.mp3")))
    root.destroy()
    return filename

In [170]:
#programme principal :
repertoireint="C:\\Users\\hp\\TDM\\Audios"
file=explort(repertoireint)   #ouvrir audio(wave)
(strg,audio_as_np_int32,framerate,bitsPerSample,sampwidth,nchannels,mn,mx,taille_wave)=préparation_data_wave(file)
compress_lzw=compress(strg)
taille_irm=compression_irm(compress_lzw,framerate,bitsPerSample,sampwidth,nchannels,mn,mx)
données_décompressées=decompression_irm()
print("La taille de l'audio original est : ",taille_wave," octets")
print("La taille de l'audio compressé est : ",taille_irm," octets")
print("Le taux de compression est :",taux_compression(taille_wave,taille_irm)," %")


La taille de l'audio original est :  320816  octets
La taille de l'audio compressé est :  180387  octets
Le taux de compression est : 43.77  %


### Ecouter l'audio compressé  (.irm) :

In [173]:
écouter_wave(données_décompressées,framerate)

### Ecouter l'audio original (.wav):

In [172]:
écouter_irm(audio_as_np_int32,framerate)
